**Load the required libraries and data**

In [1]:
# Load the required libraries 
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
import seaborn as sns 

In [ ]:
# Load the data files 
